In [1]:
import os
if os.name == 'nt':
    os.environ["OMP_NUM_THREADS"] = '2'
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../')
from utiliT.io import read_pickle, dump_pickle
from scripts.instance import *
from scripts.benders import *
import gc
from tqdm.notebook import tqdm

In [2]:
test_file = 'multipeak_100_150_400.pkl'
instance = read_pickle(f"../data/{test_file}")

In [3]:
MC_cut = dropout_cut(instance, "kmeans")
gc.collect()
MC_cut

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-20


{'method': 'kmeans-ND-dropout',
 'obj_val': 29339.596694233256,
 'n_cuts': 246,
 'n_iterations': 16,
 'avg_mp_solve': 0.004403114318847656,
 'avg_benders_loop_solve': 0.7492969781160355,
 'status': 'optimal',
 'primal_gap': 0,
 'primal_gap_perc': 0,
 'runtime': 17.048032522201538,
 'n1': 100,
 'n2': 150,
 'm1': 200,
 'm2': 300,
 'k': 400,
 'distribution': 'multipeak'}

In [4]:
MC_cut = dropout_cut(instance, "hierarchical")
gc.collect()
MC_cut

{'method': 'hierarchical-ND-dropout',
 'obj_val': 29307.09908368546,
 'n_cuts': 237,
 'n_iterations': 16,
 'avg_mp_solve': 0.004444479942321777,
 'avg_benders_loop_solve': 0.7335944920778275,
 'status': 'optimal',
 'primal_gap': 0,
 'primal_gap_perc': 0,
 'runtime': 13.866183996200562,
 'n1': 100,
 'n2': 150,
 'm1': 200,
 'm2': 300,
 'k': 400,
 'distribution': 'multipeak'}

In [5]:
MC_cut = dropout_cut(instance, "spectral")
gc.collect()
MC_cut

{'method': 'spectral-ND-dropout',
 'obj_val': 29326.991038011136,
 'n_cuts': 203,
 'n_iterations': 13,
 'avg_mp_solve': 0.004767491267277644,
 'avg_benders_loop_solve': 0.7485595666445218,
 'status': 'optimal',
 'primal_gap': 0,
 'primal_gap_perc': 0,
 'runtime': 11.751529216766357,
 'n1': 100,
 'n2': 150,
 'm1': 200,
 'm2': 300,
 'k': 400,
 'distribution': 'multipeak'}

In [6]:
MC_cut = dropout_cut(instance, "affinity")
gc.collect()
MC_cut

{'method': 'affinity-ND-dropout',
 'obj_val': 29358.030241620858,
 'n_cuts': 390,
 'n_iterations': 12,
 'avg_mp_solve': 0.006994326909383138,
 'avg_benders_loop_solve': 1.534258484840393,
 'status': 'optimal',
 'primal_gap': 0,
 'primal_gap_perc': 0,
 'runtime': 20.740079879760742,
 'n1': 100,
 'n2': 150,
 'm1': 200,
 'm2': 300,
 'k': 400,
 'distribution': 'multipeak'}

In [7]:
MC_cut = multi_cut(instance)
gc.collect()
MC_cut

{'method': 'multi-cut',
 'obj_val': 29390.251786471617,
 'n_cuts': 2762,
 'n_iterations': 9,
 'avg_mp_solve': 0.07767849498324925,
 'avg_benders_loop_solve': 15.444093916151258,
 'status': 'optimal',
 'primal_gap': 0,
 'primal_gap_perc': 0,
 'runtime': 140.25501990318298,
 'n1': 100,
 'n2': 150,
 'm1': 200,
 'm2': 300,
 'k': 400,
 'distribution': 'multipeak'}

In [8]:
MC_cut = single_cut(instance)
gc.collect()
MC_cut

{'method': 'single-cut',
 'obj_val': 29390.157893106712,
 'n_cuts': 62,
 'n_iterations': 62,
 'avg_mp_solve': 0.001829201175320533,
 'avg_benders_loop_solve': 4.804071745564861,
 'status': 'timelimit',
 'primal_gap': 0.11616695873817662,
 'primal_gap_perc': 3.952557153332683e-06,
 'runtime': 300.9297933578491,
 'n1': 100,
 'n2': 150,
 'm1': 200,
 'm2': 300,
 'k': 400,
 'distribution': 'multipeak'}

In [9]:
del MC_cut
gc.collect()

0

In [ ]:
files = os.listdir("../data")
for file in tqdm(files):
    instance_name = file[:-4]
    instance = read_pickle(f"../data/{file}")
    result = dropout_cut(instance, "kmeans")
    result['instance_name'] = instance_name
    file_name = f"../results/{result['instance_name']}_{result['method']}.pkl"
    dump_pickle(result, file_name)

In [ ]:
files = os.listdir("../data")
for file in tqdm(files):
    instance_name = file[:-4]
    instance = read_pickle(f"../data/{file}")
    result = dropout_cut(instance, "affinity")
    result['instance_name'] = instance_name
    file_name = f"../results/{result['instance_name']}_{result['method']}.pkl"
    dump_pickle(result, file_name)

In [ ]:
files = os.listdir("../data")
for file in tqdm(files):
    instance_name = file[:-4]
    instance = read_pickle(f"../data/{file}")
    result = dropout_cut(instance, "hierarchical")
    result['instance_name'] = instance_name
    file_name = f"../results/{result['instance_name']}_{result['method']}.pkl"
    dump_pickle(result, file_name)

In [ ]:
files = os.listdir("../data")
for file in tqdm(files):
    instance_name = file[:-4]
    instance = read_pickle(f"../data/{file}")
    result = dropout_cut(instance, "spectral")
    result['instance_name'] = instance_name
    file_name = f"../results/{result['instance_name']}_{result['method']}.pkl"
    dump_pickle(result, file_name)

In [ ]:
files = os.listdir("../data")
for file in tqdm(files):
    instance_name = file[:-4]
    instance = read_pickle(f"../data/{file}")
    result = multi_cut(instance)
    result['instance_name'] = instance_name
    file_name = f"../results/{result['instance_name']}_{result['method']}.pkl"
    dump_pickle(result, file_name)

In [ ]:
files = os.listdir("../data")
for file in tqdm(files):
    instance_name = file[:-4]
    instance = read_pickle(f"../data/{file}")
    result = single_cut(instance)
    result['instance_name'] = instance_name
    file_name = f"../results/{result['instance_name']}_{result['method']}.pkl"
    dump_pickle(result, file_name)